In [1]:
import re
import pandas as pd
from tqdm import tqdm

# Finding all Unique words

In [10]:
with open('big.txt','r') as fd:
    lines=fd.readlines()
    words=[]
    for line in lines:
        words+=re.findall(r'\w+',line.lower())
        
print(len(words))
vocab = list(set(words))
print(len(vocab)) 

1115585
32198


# Probability distrubtion

In [26]:
word_prob={}
for word in tqdm(vocab):
    word_prob[word]=words.count(word)/len(words)

100%|████████████████████████████████████████████████████████████████████████████| 32198/32198 [07:58<00:00, 67.28it/s]


# Text Preprocessing

## Spliting

In [50]:
def split(word):
    split_word=[]
    for i in range(len(word)+1):
        split_word+=[(word[:i],word[i:])]
    return split_word
split("Raja")

[('', 'Raja'), ('R', 'aja'), ('Ra', 'ja'), ('Raj', 'a'), ('Raja', '')]

## Delete

In [74]:

def delete(word):
    split_word=split(word)
    output=[]
    for i in split_word:
        output.append(i[0]+i[1][1:])
    return output


In [76]:
delete("lave")

['ave', 'lve', 'lae', 'lav', 'lave']

## Swap

In [96]:
def swap(word):
    output=[]
    split_word=split(word)
    for l,r in split_word:
        if len(r)>1:
            output.append(l+r[1]+r[0]+r[2:])
    return output

In [98]:
swap("Lave")

['aLve', 'Lvae', 'Laev']

## Replace

In [141]:
def replace(word):
    
    split_word=split(word)
    alphabates='abcdefghijklmnopqrstuvwxyz'
    output=[]
    for l,r in split_word:
        for char in alphabates:
            output.append(l+char+r[1:])
    return output

len(replace("Lave"))
replace("lave")

## Add 

In [156]:
def insert(word):
    
    split_word=split(word)
    alphabates='abcdefghijklmnopqrstuvwxyz'
    output=[]
    for l,r in split_word:
        for char in alphabates:
            output.append(l+char+r[0:])
    return output

In [164]:
len(insert("Lave"))

130

# Finding the prediction (Level 1)

## Combing the whole word

In [170]:
def edit(word):
    return list(set(delete(word)+replace(word)+swap(word)+insert(word)))

In [172]:
len(edit("Lave"))

236

## Predication the word

In [235]:
def spell_checking_level_1(word,count=5):
    
    suggested_word=edit(word)
    output=[]
    for word in suggested_word:
        if word in word_prob.keys():
            output.append([word,word_prob[word]])
    return  list(pd.DataFrame(output,columns=["word","prob"]).sort_values(by='prob',ascending=False).head(count)['word'].values)


In [237]:
spell_checking_level_1("lave")

['have', 'love', 'gave', 'leave', 'late']

# Finding the prediction (Level 2)

In [249]:
def spell_checking_level_2(word, count = 5):
    
    output = []
    suggested_word= edit(word)       # Level one 
    
    for e1 in edit(word):
        suggested_word += edit(e1)    # Second Level 
    
    suggested_word = list(set(suggested_word))
    
    for wrd in suggested_word:
        if wrd in word_prob.keys():
            output.append([wrd, word_prob[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)


In [251]:
        
spell_checking_level_2('fameli')

['family', 'namely', 'fame', 'amelie', 'camel']